<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/2021%20/%2004%20/%2003%20/%20%EC%9C%A0%EC%82%AC%EB%8F%84%20%EA%B2%80%EC%A6%9D_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 패키지 install

In [3]:
!pip install pymysql


# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)



#한국어 문장 분리모델
!pip install kss


!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


#mecab 코랩서 오류 안나게!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchp

In [2]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm
from konlpy.tag import Mecab
import pymysql

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')



# # mysql용 데이터 가져오기
# db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

# curs = db.cursor(pymysql.cursors.DictCursor)  

# query = '''
# select *
#     from books_data'''
# df_books_data = pd.read_sql(query, db)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g4_MoDG_6t5xuKE5yWgxkQQ8ZoY0fOLrm1PyWxnB8LYLNxS-oQUvyk
Mounted at /content/drive


In [ ]:
# 구글 드라이브

path = '/content/drive/MyDrive/'
df_books = pd.read_excel(os.path.join(path, 'book_fin.xlsx'))

In [ ]:
df_books

,Unnamed: 0,index,book_id,title,genre,words,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,0,0,0,요리코를 위해,추리/미스터리,리코 수 이자 위 누구 출발 탐정 사체 심판 무대 남자 진실 딸 패배 세팅 파악 괴...,NaN,NaN,1
1,1,1,1,그녀는 다 계획이 있다,추리/미스터리,미래 복고 동료 교코 호텔 원대 미스터리 직장 컴 자신 계획 터치 날 감사 파티 살...,NaN,NaN,1
2,2,2,2,블랙 쇼맨과 이름 없는 마을의 살인,추리/미스터리,소식 통화 살 순간 너 결혼식 시작 취직 통 약혼자 전화 경 생전 결혼 마지막 고향...,NaN,NaN,1
3,3,3,3,돌이킬 수 없는 약속,추리/미스터리,소설 수 야쿠 가쿠 전 작가 미스터리 죄 포상 놈 딸 제 란 마루 숨 추리 약속 주...,NaN,NaN,1
4,4,4,4,루팡의 딸,추리/미스터리,이 발생 카즈마 사체 자신 딸 코 날 대부 변 경찰 손녀 오 괴한 신분 도둑 발견 ...,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
2484,2484,2484,3617,드래건 살인사건,추리/미스터리,본성 풀 작품 알 구성 반 지적 분석 대 파일 미스터리 심리 내면 주말 특유 인우드...,NaN,NaN,1
2485,2485,2485,3618,가든 살인사건,추리/미스터리,상류층 본성 실내 내기 작품 경마 구성 반 가든 지적 분석 대 파일 미스터리 심리 ...,NaN,NaN,1
2486,2486,2486,3619,카지노 살인사건,추리/미스터리,불리 피해자 상류층 어디 맨해튼 미심 발생 꼬리 케이드 보이 미국 물 파일 몬테카를...,NaN,NaN,1
2487,2487,2487,3620,단테클럽 1,추리/미스터리,다빈치 활동 과정 작품 소설 번역 저자 미국 공로 실화 협회 단테 제작 붐 신곡 살...,NaN,NaN,1


In [ ]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73508 entries, 0 to 73507
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  73508 non-null  int64  
 1   title       73508 non-null  object 
 2   author      64865 non-null  object 
 3   pub         73508 non-null  object 
 4   price       73508 non-null  int64  
 5   review      73508 non-null  int64  
 6   rating      73508 non-null  float64
 7   story       67630 non-null  object 
 8   img         65530 non-null  object 
 9   genre       73508 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.6+ MB


# 유사도 검증 

In [4]:
# 유사도 검증에 필요한 패키지 import
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# words(스토리)에 대해서 tf_idf 수행
tfidf = TfidfVectorizer()
tfidf_matrix_mys = tfidf.fit_transform(df_books['words'])
print(tfidf_matrix_mys.shape)

# 유사도 진행
cosine_matrix_mys = cosine_similarity(tfidf_matrix_mys, tfidf_matrix_mys)

# 결과에 얻고자 하는 책 타이틀 리스트
books_mys_id = {}
for i, c in enumerate(df_books['words']): books_mys_id[i] = c
        
# 결과에 얻고자 하는 책 id 리스트
id2books_mys = {}
for i, c in books_mys_id.items(): id2books_mys[c] = i


def get_recommendations(title, cosine_sim=cosine_matrix_mys):

    idx = id2books_mys[title]
    sim_scores_mys = [(i,c) for i, c in enumerate(cosine_matrix_mys[idx]) if i !=idx] 
    sim_scores_mys = sorted(sim_scores_mys, key =lambda x: x[1], reverse = True)
    sim_scores_mys[0:10]
    

    sim_scores_mys = [(books_mys_id[i], score) for i, score in sim_scores_mys[0:10]]
    return sim_scores_mys

NameError: ignored

In [ ]:
# 유사도 검증하고 하고자하는 책 입력
get_recommendations('돌이킬 수 없는 약속')

[('단테 클럽 2', 0.9744196724179254),
 ('단테 클럽 1', 0.9744196724179254),
 ('단테클럽 2', 0.256154929588162),
 ('로스트 심벌 2', 0.16397715030813564),
 ('공개수배', 0.16034001007004794),
 ('로스트 심벌 1', 0.159380637654646),
 ('살인게임', 0.15849356786754487),
 ('�A빠이, 이상', 0.1437867786664614),
 ('롱기누스의 창', 0.14301579936497277),
 ('4의 규칙 2', 0.14301080471633362)]

(2489, 9006)
